# Ideas for the Project

Only Scrape from "Data Scientist" and "Data Analyst" DO NOT include Business Analyst or prefixes like Senior/Junior

## Features 
-Number of Required Programming Languages
-Education Level
-Years of Experience
-Salary 
-Benefits (401K and Healthcare ex)

In [49]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import nltk
import numpy as np
import re
from nltk.corpus import wordnet
from selenium import webdriver

url = 'https://www.linkedin.com/jobs/view/3483258935/?alternateChannel=search&refId=%2Bn7M6PBK5H8QlAYUp%2BKiMA%3D%3D&trackingId=31KIGxJZHoj5eOdy2VaF6g%3D%3D'
test = requests.get(url)
test

<Response [200]>

In [11]:
test = BeautifulSoup(test.text)
print(test.prettify())

<Response [200]>

In [12]:
test = test.find_all('div', {'class':"show-more-less-html__markup show-more-less-html__markup--clamp-after-5"})
test

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta content="d_jobs_guest_details" name="pageKey"/>\n  <!-- -->\n  <meta content="en_US" name="locale"/>\n  <meta data-app-version="2.0.1256" data-browser-id="37b80c60-fdc9-4c8c-8e52-161f5687e169" data-call-tree-id="AAX1FFaiST+Ska0MCGIemQ==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_details;vJ6MOkvXQSSybhB0C6lPQw==" data-service-name="jobs-guest-frontend" id="config"/>\n  <link href="https://www.linkedin.com/jobs/view/data-scientist-intern-at-yahoo-3407056800" rel="canonical"/>\n  <!-- -->\n  <!-- -->\n  <!-- -->\n  <!-- -->\n  <meta content="https://www.linkedin.com/jobs/view/data-scientist-intern-at-yahoo-3407056800" property="al:android:url"/>\n  <meta content="com.linkedin.android" property="al:android:package"/>\n  <meta content="LinkedIn" property="al:android:app_name"/>\n  <met

In [ ]:
Doc = test[0].text
Doc

In [41]:
def raw_text(url):
    """Takes a URL as input and performs web scrapping to retrieve the body of the
    webpage (in this case a Linkedin ad)"""
    ad = requests.get(url) #Retrieve webpage
    Html = BeautifulSoup(ad.text, 'html.parser') #Convert html into a nicer format
    text_body = Html.find_all('div', 
                              {'class':"show-more-less-html__markup show-more-less-html__markup--clamp-after-5"})
    text_body = text_body[0].text
    return text_body
def clean_text(doc):
    """Take an unstructured document and tokenize it into a list of words. 
    Then standardize it by lowercasing and lemmatizing each word"""
    words = re.findall(r'(?:[a-zA-Z]|#|"+")+',doc) #Find all alphabetical words (Preserve + and # for C++ and C#)
    stopwords = nltk.corpus.stopwords.words("english")
    words = [i.lower() for i in words] #Lowercase all words
    words = [i for i in words if i not in stopwords] #Filter out stopwords
    tag_words = nltk.pos_tag(words) #Begin lemmatizing by tagging each word
    tag_words = [(i, wordnet_pos(j)) for (i, j) in tag_words] #Convert the tags into something the lemmatizer understands
    lemmatizer = nltk.WordNetLemmatizer()
    clean_words = [lemmatizer.lemmatize(i, j) for i, j in tag_words] #Lemmatize the words
    #Document should be cleaned up
    return clean_words
def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN) #Function created by Bo Ning inWeek 6-2
def lang_count(text):
    """ Take a body of clean text and count the number of programming languages present"""
    languages = ['python','r','sql','sa','c',
                 'c++','c#','java','javascript',
                 'julia','matlab','swift','tableau'
                'microsoft'] #SAS turns into sa after lemmatization
    count = sum([i in text for i in languages]) #Check if each language is in the ad
    #And sum the number of programming languages present
    return count

In [42]:
raw = raw_text(url)

In [43]:
clean = clean_text(raw)

In [44]:
clean

['top',
 'skill',
 'detail',
 'extensive',
 'background',
 'data',
 'scientist',
 'work',
 'within',
 'insurance',
 'industry',
 'advance',
 'knowledge',
 'data',
 'analysis',
 'tool',
 'data',
 'visualization',
 'develop',
 'analysis',
 'query',
 'procedures',
 'sql',
 'sa',
 'bi',
 'tool',
 'analysis',
 'software',
 'relevant',
 'industry',
 'data',
 'method',
 'ability',
 'connect',
 'external',
 'insight',
 'business',
 'problem',
 'experience',
 'predictive',
 'modeling',
 'ml',
 'machine',
 'learn',
 'code',
 'build',
 'decision',
 'model',
 'logistic',
 'regression',
 'decision',
 'tree',
 'etc',
 'prefer',
 'required',
 'bachelor',
 'degree',
 'quantitative',
 'analytics',
 'field',
 'economics',
 'finance',
 'statistic',
 'mathematics',
 'actuarial',
 'science',
 'operation',
 'research',
 'data',
 'business',
 'analysis',
 'data',
 'science',
 'quantitative',
 'discipline',
 'ability',
 'independently',
 'formulate',
 'hypothesis',
 'work',
 'towards',
 'prove',
 'disprove',


In [45]:
lang_count(clean)

2

In [46]:
raw

'\n        Top Skills\' Details  Extensive background as a Data Scientist working within the insurance industry Advanced knowledge of data analysis tools, data visualization, developing analysis queries and procedures in SQL, SAS, BI tools or other analysis software, and relevant industry data & methods and ability to connect external insights to business problems. Experience with Predictive Modeling and ML (machine learning) code to build decision models such as logistic regression, decision trees, etc. Preferred But Not Required  Bachelor\'s degree in quantitative analytics field such as Economics, Finance, Statistics, Mathematics, Actuarial Sciences, Operations Research, Data and/or Business Analysis, Data Science or other quantitative discipline Ability to independently formulate hypotheses and work towards proving/disproving assumptions. Ability to present data-based evidence to inform and influence senior leadership. Description Our client, is looking for a strong Data Scientist 

In [62]:
url2 = 'https://www.linkedin.com/jobs/view/3480666936/?alternateChannel=search&refId=t60HhiMPmHHA%2BECek30NHg%3D%3D&trackingId=RP8oT%2BfKL5nI151FDHwofA%3D%3D'


In [63]:
test2 = requests.get(url2)
test2

<Response [200]>

In [64]:
salary = BeautifulSoup(test2.text,'html.parser')

In [65]:
salary

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="d_jobs_guest_details" name="pageKey"/>
<!-- --> <meta content="en_US" name="locale"/>
<meta data-app-version="2.0.1256" data-browser-id="b4a9b539-ea78-4b38-8e87-b9ef24b2f0fe" data-call-tree-id="AAX1eLhaUkPmtDhRxH9L7A==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_details;B6rFouhmQqetdbSlQSXRLg==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<meta content="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" property="al:android:url"/>
<meta content="com.linkedin.android" property="al:android:package"/>
<meta content="LinkedIn" property="al:android:app_name"/>
<meta content="ht

In [39]:
test2.text

'<!DOCTYPE html>\n\n    \n    \n    \n    \n    \n\n    \n    <html lang="en">\n      <head>\n        <meta name="pageKey" content="d_jobs_guest_details">\n<!---->        <meta name="locale" content="en_US">\n        <meta id="config" data-app-version="2.0.1256" data-call-tree-id="AAX1eErPKplTcutF3hDXvQ==" data-multiproduct-name="jobs-guest-frontend" data-service-name="jobs-guest-frontend" data-browser-id="cf3c6083-8d54-4ee6-8abb-03d04b2d63eb" data-enable-page-view-heartbeat-tracking data-page-instance="urn:li:page:d_jobs_guest_details;l0QOX86xQxapK96WdAGGjA==" data-disable-jsbeacon-pagekey-suffix="false" data-member-id="0">\n\n        <link rel="canonical" href="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936">\n<!----><!---->\n<!---->\n<!---->\n          <meta property="al:android:url" content="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936">\n          <meta property="al:android:package" content

In [40]:
salary

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="d_jobs_guest_details" name="pageKey"/>
<!-- --> <meta content="en_US" name="locale"/>
<meta data-app-version="2.0.1256" data-browser-id="cf3c6083-8d54-4ee6-8abb-03d04b2d63eb" data-call-tree-id="AAX1eErPKplTcutF3hDXvQ==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_details;l0QOX86xQxapK96WdAGGjA==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<meta content="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" property="al:android:url"/>
<meta content="com.linkedin.android" property="al:android:package"/>
<meta content="LinkedIn" property="al:android:app_name"/>
<meta content="htt

In [30]:
salary.find_all('div',{'class':"jobs-unified-description__salary-main-rail-card"})

[]

In [16]:
raw_text(url2)

'\nAs a Data Scientist in our Analytic Solutions team, you will contribute to the firm’s objectives by delivering rapid and scalable solutions that unlock value for Cerberus desks, portfolio companies, or other businesses/investments. You’ll do this by designing, implementing, and deploying (1) machine learning models, (2) data visualizations & business intelligence, and (3) insights & analytics for a broad range of business objectives. You may also participate in due diligence analyses of future investments and evaluate 3rd party solutions and cloud-based tools for client adoption.  Responsibilities: Develop, plan, and execute analytical projects as an individual contributor and in teamsDelivery focused: Help design analytical and technical solutions using a rigorous hypothesis-based approach, partner with technical teams to build a development plan, and then help execute the development and implementation with a focus on high-impact returnAgile and pragmatic: Ability to rapidly deliv

In [58]:
browser = webdriver.Chrome()

In [59]:
browser.get(url2)

In [60]:
html = browser.execute_script('return document.documentElement.outerHTML')

In [61]:
BeautifulSoup(html,'html.parser')

<html lang="en"><head>
<meta content="d_jobs_guest_details" name="pageKey"/>
<!-- --> <meta content="en_US" name="locale"/>
<meta data-app-version="2.0.1256" data-browser-id="0d19a016-6969-4b58-80ce-466acee88bdf" data-call-tree-id="AAX1eKzORwcI7UaxNF2Y+Q==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_details;K1qkjHs/So62zc3q4Z5+qw==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<meta content="https://www.linkedin.com/jobs/view/data-scientist-at-cerberus-capital-management-3480666936" property="al:android:url"/>
<meta content="com.linkedin.android" property="al:android:package"/>
<meta content="LinkedIn" property="al:android:app_name"/>
<meta content="https://www.linkedin